# Lab 10: Text Mining

This lab material is largely self-contained. We assume that every student has already taken STAT7008 or knows some basic operations of Python. Noet that you may use Anaconda to run the .ipynb file. For the installation of Anaconda, please see https://conda.io/docs/user-guide/install/index.html.

### Purpose

In Lab 10, you will learn how to:

a. do spam detection.

### Useful libraries for this Lab

a. numpy, for data array. 

b. sklearn, for preprocessing and modelling.

c. os, for the working directory. 

In [1]:
import numpy as np
import pandas as pd
import email.parser 
import os, sys, stat
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression as lr
import shutil
wd = os.getcwd() # Set your working directory. 
print wd

/home/renjielu/PycharmProjects/DM8017/DM_Lab10


The data used in Lab 10 is one of the datasets for the data mining competition associated with ICONIP 2010, see http://csmining.org/index.php/spam-email-datasets-.html. We only use the traning data since no label provided in the test data. In the training set, there are 4327 messages, containing 2949 non-spam messages (HAM) and 1378 spam messagees (SPAM). 

The following part is to extract all original files to the list varialbe all_emails.

In [3]:
#!/usr/bin/python
# FileName: Subsampling.py provided by the official website. Version 1.0 by Tao Ban, 2010.5.26
# This function extract all the contents, ie subject and first part from the .eml file 
# and store it in a new file with the same name in the dst dir. 

def ExtractSubPayload (filename):
    ''' Extract the subject and payload from the .eml file.
    '''
    if not os.path.exists(filename): # dest path doesnot exist
        print "ERROR: input file does not exist:", filename
        os.exit(1)
    fp = open(filename)
    msg = email.message_from_file(fp)
    payload = msg.get_payload()
    if type(payload) == type(list()) :
        payload = payload[0] # only use the first part of payload
    sub = msg.get('subject')
    sub = str(sub)
    if type(payload) != type('') :
        payload = str(payload)
    
    return sub + payload

def ExtractBodyFromDir ( srcdir, dstdir ):
    '''Extract the body information from all .eml files in the srcdir and 

    save the file to the dstdir with the same name.'''
    if not os.path.exists(dstdir): # dest path doesnot exist
        os.makedirs(dstdir)  
        
    files = os.listdir(srcdir)
    for file in files:
        srcpath = os.path.join(srcdir, file)
        dstpath = os.path.join(dstdir, file)
        src_info = os.stat(srcpath)
        if stat.S_ISDIR(src_info.st_mode): # for subfolders, recurse
            ExtractBodyFromDir(srcpath, dstpath)
        else:  # copy the file
            body = ExtractSubPayload (srcpath)
            dstfile = open(dstpath, 'w')
            dstfile.write(body)
            dstfile.close()


###################################################################
# main function start here
# srcdir is the directory where the .eml are stored

srcdir = wd+'/TRAINING'
if not os.path.exists(srcdir):
    print 'The source directory %s does not exist, exit...' % (srcdir)
    sys.exit()
# dstdir is the directory where the content .eml are stored

dstdir = wd+'/Training'
if not os.path.exists(dstdir):
    print 'The destination directory is newly created.'
    os.makedirs(dstdir)

###################################################################
ExtractBodyFromDir ( srcdir, dstdir ) 

# put all emails into one list. 

from bs4 import BeautifulSoup

dirs = os.listdir(dstdir)

all_emails = []
    
for fname in dirs:
    f = open(dstdir + '/' + fname, 'r')
    html = f.read()
    f.close()
    soup = BeautifulSoup(html, 'html.parser')
    s = soup.get_text()
    
    all_emails.append(s)

print len(all_emails)

4327


We show the content of the second email as follows. 

We build a class called vectorizer by "CountVectorizer" in Sklearn. It converts a collection of text documents to the so-called document-term frequency matrix (DTM). Note that this is the transpose of the term-document frequency matrix defined in the lecture note. The binary DTM can be obtained by setting the argument "binary" in CountVectorizer as True. Its default value is False.

Due to the memory limit in the laptop, we only consider the top 10000 features ordered by term frequency across the corpus (created by CountVectorizer). The stop word list we used here is 'english', which is a built-in word list in Sklearn. 

According to the training data, we build the corpus, and print the first eight elements of it.

We transform the transformed data to numpy arrays by calling the toarray() function, see below. 

In [4]:
# the second email in the data.

print all_emails[1]

# 

vectorizer = CountVectorizer(stop_words='english', max_features=10000)

vectorizer.fit(all_emails)

print '-------------------------------------------------'
print '-------------------------------------------------'
print vectorizer.vocabulary_.keys()[:8]

dt_matrix = vectorizer.transform(all_emails)

print dt_matrix.shape
DTM = dt_matrix.toarray()

[ILUG] [OT]:Bill Gates in PeruHi guys,
   slightly OT. This comes from The Register from last
month, but I only came across it;

http://www.theregister.co.uk/content/4/26207.html

   Some/all of you may know that the government in
Peru were going to instruct state bodies to look at
using open source software in an attempt to save money
over M$ licences. Well, read the above article.

   Eamonn

__________________________________________________
Do You Yahoo!?
Everything you'll ever need on one web page
from News and Sport to Email and Music Charts
http://uk.my.yahoo.com

-- 
Irish Linux Users' Group: ilug@linux.ie
http://www.linux.ie/mailman/listinfo/ilug for (un)subscription information.
List maintainer: listmaster@linux.ie


-------------------------------------------------
-------------------------------------------------
[u'hanging', u'woody', u'bringing', u'wednesday', u'errors', u'hp', u'usenet', u'designing']
(4327, 10000)


Based on DTM above, we compute the inverse document frequency matrix by "TfidfTransformer". Note that "TfidfVectorizer" in Sklearn can give the inverse document frequency matrix directly. 

In [6]:
IDF = TfidfTransformer(norm = None)

IDFM = IDF.fit_transform(DTM)
IDFM = IDFM.toarray()

print IDFM[:2, :88]


[[ 0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   2.85666774  0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   3.50639276  0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          3.44593479  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          4.27160359  0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.        ]
 [ 0.        

Finally, we build two logistic regression models by using DTM and IDFM, respectively. The results show that the accuracy from the latter model is slightly higher. 

In [8]:

all_labels = []

f = open(wd+'/'+'SPAMTrain.label')
tmp = f.read()
f.close()

tmp = tmp.split('\n')
tmp_ = [x.split(' ')[0] for x in tmp]

tmp_.remove('')

labels = np.array([int(x) for x in tmp_])

lrg = lr()

lrg.fit(DTM, labels)

print 'the accuracy computed from CM: ', lrg.score(DTM, labels)


lrg.fit(IDFM, labels)

print 'the accuracy computed from IDFM: ', lrg.score(IDFM, labels)

the accuracy computed from CM:  0.914028195054
the accuracy computed from IDFM:  0.967876126647
